In [119]:
import dotenv
import os

dotenv.load_dotenv("../../")
polygon_api_key = os.environ["POLYGON_API_KEY"]
assert polygon_api_key

In [120]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)

In [121]:
px = client.get_aggs(
    ticker="AAPL", multiplier=1, timespan="day", from_="2022-03-01", to="2023-03-13"
)

In [122]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame

class OptionSpec:
    def __init__(
        self,
        und_price: float,
        strike: float,
        yte: float,
        rfr: float,
        market_quote: float,
    ) -> None:
        S, K, T, R = und_price, strike, yte, rfr
        self.und_price = und_price
        self.strike = strike
        self.yte = yte
        self.rfr = rfr

        def d1(S, K, T, r, sigma):
            return (log(S / K) + (r + sigma**2 / 2.0) * T) / (sigma * sqrt(T))

        def d2(S, K, T, r, sigma):
            return d1(S, K, T, r, sigma) - sigma * sqrt(T)

        def call_implied_volatility(Price, S, K, T, r):
            sigma = 0.001
            while sigma < 1:
                Price_implied = S * norm.cdf(d1(S, K, T, r, sigma)) - K * exp(
                    -r * T
                ) * norm.cdf(d2(S, K, T, r, sigma))
                if Price - (Price_implied) < 0.001:
                    return sigma
                sigma += 0.001
            return 0

        def put_implied_volatility(Price, S, K, T, r):
            sigma = 0.001
            while sigma < 1:
                Price_implied = K * exp(-r * T) - S + S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
                if Price - (Price_implied) < 0.001:
                    return sigma
                sigma += 0.001
            return 0

        c_sigma = call_implied_volatility(Price=market_quote, S=S, K=K, T=T, r=R)
        p_sigma = put_implied_volatility(Price=market_quote, S=S, K=K, T=T, r=R)
        self.c_iv = c_sigma
        self.p_iv = p_sigma

        if c_sigma != 0:
            self.call_price = S * norm.cdf(d1(S, K, T, R, c_sigma)) - K * exp(
                -R * T
            ) * norm.cdf(d2(S, K, T, R, c_sigma))
            self.put_price = K * exp(-R * T) - S * self.call_price

            self.c_delta = norm.cdf(d1(S, K, T, R, c_sigma))
            self.c_gamma = norm.pdf(d1(S, K, T, R, c_sigma)) / (S * c_sigma * sqrt(T))
            self.c_vega = 0.01 * (S * norm.pdf(d1(S, K, T, R, c_sigma)) * sqrt(T))
            self.c_theta = 0.01 * (
                -(S * norm.pdf(d1(S, K, T, R, c_sigma)) * c_sigma) / (2 * sqrt(T))
                - R * K * exp(-R * T) * norm.cdf(d2(S, K, T, R, c_sigma))
            )
        
        if p_sigma != 0:
            self.p_delta = -norm.cdf(-d1(S, K, T, R, p_sigma))
            self.p_gamma = norm.pdf(d1(S, K, T, R, p_sigma)) / (S * p_sigma * sqrt(T))
            self.p_vega = 0.01 * (S * norm.pdf(d1(S, K, T, R, p_sigma)) * sqrt(T))
            self.p_theta = 0.01 * (
                -(S * norm.pdf(d1(S, K, T, R, p_sigma)) * p_sigma) / (2 * sqrt(T))
                + R * K * exp(-R * T) * norm.cdf(-d2(S, K, T, R, p_sigma))
            )
    

In [123]:
px[0].timestamp

1646110800000

In [124]:
def build_polygon_ticker(ticker: str, strike: float, expiry: date, call_put = "C"):
    strike_str = str(strike).split('.')
    wholes = strike_str[0].rjust(5, '0')
    decimals = strike_str[1].rjust(3, '0') if len(strike_str) > 1 else "000"
    return (
        f"O:{ticker}"
        f"{expiry.year%100}{str(expiry.month).rjust(2,'0')}{str(expiry.day).rjust(2,'0')}"
        f"{call_put}"
        f"{wholes}{decimals}"
    )

In [125]:
from pymongo import MongoClient
def get_database():
 
   # Provide the mongodb atlas url to connect python to mongodb using pymongo
   CONNECTION_STRING = "mongodb://localhost:27017"
 
   # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
   client = MongoClient(CONNECTION_STRING)
 
   # Create the database for our example (we will use the same database throughout the tutorial
   return client['quant']
db = get_database()
opts_db = db.get_collection("opts")
und_px_db = db.get_collection("und_px")

In [127]:
call_ticker = build_polygon_ticker("AAPL", 150, date(2023, 12, 15), "C")
put_ticker = build_polygon_ticker("AAPL", 150, date(2023, 12, 15), "P")
import datetime
import time
base_day = date(2022, 1, 7)
und_ticker = "AAPL"

for strike in range(120,180,5):
    curr_day = base_day
    while curr_day < date.today():
        try:
            curr_day = curr_day + datetime.timedelta(days=7)
            time.sleep(13)
            call_ticker = client.get_aggs(
                ticker=build_polygon_ticker(und_ticker, strike, curr_day, "C"), 
                multiplier=1, timespan="day", from_="2022-01-01", to="2023-03-12"
            )
            d = pd.DataFrame(call_ticker).set_index("timestamp").T.to_dict()
            d = {
                str(k): v
                for (k,v) in d.items()
            }
            d['expiry_date'] = str(curr_day)
            d['strike'] = strike
            opts_db.insert_one(d)
        except Exception as e:
            print(e)

Expected key "results" in response {'ticker': 'O:AAPL220415C00120000', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '77f77797029ba68df19acc14b1722307'}.Make sure you have sufficient permissions and your request parameters are valid.This is the url that returned no results: https://api.polygon.io/v2/aggs/ticker/O:AAPL220415C00120000/range/1/day/2022-01-01/2023-03-12
Expected key "results" in response {'ticker': 'O:AAPL220415C00125000', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': '3fa0d27f2594cc5c9c046e75ae9ef173'}.Make sure you have sufficient permissions and your request parameters are valid.This is the url that returned no results: https://api.polygon.io/v2/aggs/ticker/O:AAPL220415C00125000/range/1/day/2022-01-01/2023-03-12
Expected key "results" in response {'ticker': 'O:AAPL220415C00130000', 'queryCount': 0, 'resultsCount': 0, 'adjusted': True, 'status': 'OK', 'request_id': 'e8e9f6dec7aaa5c021c070916b405e